In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
model_name = "richielo/small-e-czech-finetuned-ner-wikiann"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = """
„BIS je připravena poskytovat prezidentu republiky maximální informační podporu, tak jak je to běžné ve vyspělých demokratických státech, a to všemi formami včetně pravidelných bezpečnostních briefingů,“ citovala BIS Koudelku.

Pavlův přístup ostře kontrastuje s přístupem dosluhují hlavy státu Miloše Zemana. Ten opakovaně kritizoval BIS i samotného Koudelku, jehož odmítal jmenovat generálem.

Návrhy mu předložila vláda Andreje Babiše (ANO) i Petra Fialy (ODS). Další návrh na povýšení současná vláda již dříve avizovala na období po volbách, až se funkce ujme nový prezident. Pavel před volbami řekl, že by Koudelku do generálské hodnosti jmenoval.
"""

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<class 'list'>
[{'entity': 'B-ORG', 'score': 0.92189443, 'index': 6, 'word': 'prezident', 'start': 31, 'end': 40}, {'entity': 'I-ORG', 'score': 0.91672975, 'index': 7, 'word': '##u', 'start': 40, 'end': 41}, {'entity': 'I-ORG', 'score': 0.9792932, 'index': 8, 'word': 'republiky', 'start': 42, 'end': 51}, {'entity': 'B-LOC', 'score': 0.6451612, 'index': 21, 'word': 'demokra', 'start': 115, 'end': 122}, {'entity': 'I-LOC', 'score': 0.9821082, 'index': 22, 'word': '##tických', 'start': 122, 'end': 129}, {'entity': 'I-LOC', 'score': 0.94849604, 'index': 23, 'word': 'státech', 'start': 130, 'end': 137}, {'entity': 'B-PER', 'score': 0.9980355, 'index': 60, 'word': 'miloše', 'start': 296, 'end': 302}, {'entity': 'I-PER', 'score': 0.99940944, 'index': 61, 'word': 'zemana', 'start': 303, 'end': 309}, {'entity': 'B-ORG', 'score': 0.75892353, 'index': 67, 'word': 'bis', 'start': 336, 'end': 339}, {'entity': 'B-PER', 'score': 0.9958247, 'index': 70, 'word': 'kou', 'start': 352, 'end': 355}, {'enti

In [17]:
def filter_person_entities(entities: list):
    return [ent for ent in entities if 'PER' in ent['entity']]

def get_person_entities(text: str, ner_pipeline):
    entities = ner_pipeline(text)
    return filter_person_entities(entities)

def replacer(s, newstring, index, nofail=False):
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + 1:]

def get_masked_text(text: str, ner_pipeline):
    masked_text = text
    entities = get_person_entities(example, ner_pipeline)
    for ent in entities:
        running_index = ent.get('start')
        while True:            
            masked_text = replacer(masked_text, '*', running_index)
            running_index += 1
            if running_index == ent.get('end'):
                break
    return masked_text

print('---Original---')
print(example)
print('print('---Original---')')
print(get_masked_text(example, nlp))


„BIS je připravena poskytovat prezidentu republiky maximální informační podporu, tak jak je to běžné ve vyspělých demokratických státech, a to všemi formami včetně pravidelných bezpečnostních briefingů,“ citovala BIS Koudelku.

Pavlův přístup ostře kontrastuje s přístupem dosluhují hlavy státu Miloše Zemana. Ten opakovaně kritizoval BIS i samotného Koudelku, jehož odmítal jmenovat generálem.

Návrhy mu předložila vláda Andreje Babiše (ANO) i Petra Fialy (ODS). Další návrh na povýšení současná vláda již dříve avizovala na období po volbách, až se funkce ujme nový prezident. Pavel před volbami řekl, že by Koudelku do generálské hodnosti jmenoval.






„BIS je připravena poskytovat prezidentu republiky maximální informační podporu, tak jak je to běžné ve vyspělých demokratických státech, a to všemi formami včetně pravidelných bezpečnostních briefingů,“ citovala BIS Koudelku.

Pavlův přístup ostře kontrastuje s přístupem dosluhují hlavy státu ****** ******. Ten opakovaně kritizoval BIS i